In [1]:
import pycutestmgr as cute
import time
import pdb 
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt
import scipy.sparse as spc
from ipsolver import parse_cutest_like_problem, ipsolver, parse_matlab_like_problem, check_derivative
cute.clearCache('HS71')

# Import Problem

In [2]:
problems = [("CORKSCRW", {"T": 50}),  # 1
            ("COSHFUN", {"M": 20}),  # 2 --- many iteractions
            ("DIXCHLNV", {"N":100}), # 3
            ("HAGER4", {"N": 1000}), # 4
            ("HIMMELBK", {}), # 5
            ("NGONE", {"HNS": 49}), # 6 --- many iteractions
            ("OPTCNTRL", {}), # 7
            ("OPTMASS", {"N": 200}),  # 8 --- OPT 4.6e-4
            ("ORTHREGF", {"NPTS": 20}),  # 9
            ("SVANBERG", {"N": 500}),  # 10
            ("READING1", {"N": 100}), # 11  --- Fails !!! Warning --- Take a look
            ("HS7", {}), # 12
            ("HS10", {}), # 13
            ("HS11", {}), # 14
            ("HS13", {}), # 15
            ("HS14", {}), # 16
            ("HS16", {}), # 17
            ("HS17", {}), # 18
            ("HS19", {}), # 19
            ("HS20", {}), # 20
            ("HS22", {}), # 21
            ("HS24", {}), # 22
            ("HS26", {}), # 23
            ("HS28", {}), # 24
            ("HS31", {}), # 25
            ("HS32", {}), # 26
            ("HS33", {}), # 27
            ("HS39", {}), # 28
            ("HS46", {}), # 29
            ("HS51", {}), # 30
            ("HS52", {}), # 31
            ("HS53", {}), # 32
            ("HS63", {}), # 33
            ("HS64", {}), # 34
            ("HS65", {}), # 35
            ("HS70", {}), # 36
            ("HS71", {}), # 37
            ("HS72", {}), # 38
            ("HS73", {}), # 39
            ("HS74", {}), # 40
            ("HS75", {}), # 41
            ("HS77", {}), # 42
            ("HS78", {}), # 43
            ("HS79", {}), # 44
            ("HS80", {}), # 45
            ("HS81", {}), # 46
            ("HS83", {}), # 47
            ("HS84", {}), # 48
            ("HS85", {}), # 49 --- Fails!
            ("HS86", {}), # 50
            ("HS93", {}), # 51
            ("HS95", {}), # 52
            ("HS96", {}), # 53
            ("HS97", {}), # 54 --- Fails!
            ("HS98", {}), # 55 --- Fails!
            ("HS99", {}), # 56 --- Fails!
            ("HS100", {}), # 57
            ("HS104", {}), # 58
            ("HS105", {}), # 59 --- Fails! nan -- Take a look!!
            ("HS106", {}), # 60
            ("HS107", {}), # 61
            ("HS108", {}), # 62
            ("HS109", {}), # 63  --- Fails! Singular factor --- Take a look!!
            ("HS111", {}), # 64
            ("HS112", {}), # 65  --- Fails! nan  --- Take a look!
            ("HS113", {}), # 66
            ("HS114", {}), # 67
            ("HS116", {}), # 68  --- Doesn't Fails! nan  --- Take a look!
            ("HS117", {}), # 69 
            ("HS118", {}), # 70
            ("HS119", {}), # 71  --- Doesn't Fails! nan  --- Take a look!
            ]

In [3]:
number = 71
name = problems[number-1][0]
cute.clearCache(name)
cute.prepareProblem(name, sifParams=problems[number-1][1])
problem = cute.importProblem(name)
info = problem.getinfo()
x0 = info["x"]
n = info["n"]
m = info["m"]
print(n, m)
v0 = info["v"]
eq_list = info["equatn"]

def convert_to_inf(bounds):
    new_bounds = np.zeros_like(bounds)
    i = 0
    for b in bounds:
        if b >= 1e20:
            new_bounds[i] = np.inf
        elif b <= -1e20:
            new_bounds[i] = -np.inf
        else:
            new_bounds[i] = b
        i += 1
    return new_bounds

# Define upper and lower bound
c_lb = convert_to_inf(info["cl"])
c_ub = convert_to_inf(info["cu"])
lb = convert_to_inf(info["bl"])
ub = convert_to_inf(info["bu"])

# Define function and constraints
def fun(x):
    return problem.obj(x)[0]

def grad(x):
    _, g1 = problem.obj(x, True)
    return g1

def lagr_hess(x, v):
    def matvec(p):
        return problem.hprod(p, x, v)
    return spc.linalg.LinearOperator((n, n), matvec=matvec)


def constr(x):
    return problem.cons(x)

def jac(x):
    _, A1 = problem.cons(x, True)
    return spc.csc_matrix(A1)

# Parse constraints
fun, grad, new_lagr_hess, n_total_ineq, constr_ineq, \
    jac_ineq, n_total_eq, constr_eq, jac_eq \
        = parse_cutest_like_problem(
            fun, grad, lagr_hess, n, constr, jac,
            c_lb, c_ub, lb, ub)

(16, 8)


# Optimize Problem

In [4]:
start_time = time.time()
x, info = ipsolver(fun, grad, new_lagr_hess, n_total_ineq, constr_ineq,
                jac_ineq, n_total_eq, constr_eq, jac_eq, x0)
total_time = time.time() - start_time

0 1.0 1.0 1.00350916828 41230.0555302
1 7.0 1.0 1.07472813237 207369.135895
2 49.0 1.0 3.76458089422 76136.1097532
3 152.969278313 1.0 1.22989601404 9927.30339239
4 152.969278313 1.0 1.13010567811 2250.32031417
5 152.969278313 1.0 1.08639689722 577.028383039
6 152.969278313 1.0 1.04008848193 203.927779687
7 152.969278313 1.0 1.17302876601 88.3361714803
8 152.969278313 1.0 1.09129256496 21.7992485008
9 152.969278313 1.0 1.28232108748 2.99650786958
10 152.969278313 1.0 1.03237676079 1.14425819003
11 152.969278313 1.0 0.620090247884 0.163113077805
====> 0.019331277822 7.84060553384e-15 0.1 0.1 12
0 764.846391566 1.0 1.47497404601 0.174822473866
====> 0.0165538468151 3.64825916673e-15 0.02 0.02 1
0 3824.23195783 1.0 1.33195698763 0.034745992248
1 3824.23195783 1.0 0.410358453268 0.0193115595571
====> 0.000707748328579 1.49077891623e-14 0.004 0.004 2
0 19121.1597892 1.0 1.47560467075 0.00630724986071
====> 0.00051071389881 2.60682141937e-15 0.0008 0.0008 1
0 95605.7989458 1.0 1.19032926339 

/home/antonio/Dropbox/My Projects/ip-nonlinear-solver/ipsolver/ipsolver.py:79: RuntimeWarning: invalid value encountered in add
  self.constr_ineq(x) + s))


# Display Results

In [6]:
print('f = '+ str(info["fun"]))
print('optimality = ' + str(info["opt"]))
print('c violation = ' + str(info["constr_violation"]))
print('niter = ' + str(info["niter"]))
print('total time = ' + str(total_time))

f = 244.899718266
optimality = 6.35763669967e-08
c violation = 9.13214143349e-12
niter = 38
total time = 0.232614040375
